# Day 4: Risk Management

## Learning Objectives
- Calculate VaR and CVaR
- Implement drawdown management
- Dynamic risk controls

---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

print("✅ Libraries loaded!")
print("📚 Day 4: Risk Management")

In [ ]:
# ============================================================
# GENERATE STRATEGY RETURNS
# ============================================================

print("GENERATING STRATEGY RETURNS")
print("="*60)

np.random.seed(42)
n_days = 1000

# Simulate strategy returns with some fat tails
returns = np.random.normal(0.0003, 0.012, n_days)

# Add occasional large moves
for i in range(n_days):
    if np.random.random() < 0.02:  # 2% chance of shock
        returns[i] = np.random.choice([-1, 1]) * np.random.uniform(0.03, 0.06)

cumulative = np.cumprod(1 + returns)

print(f"Strategy statistics:")
print(f"  Mean daily return: {returns.mean()*100:.4f}%")
print(f"  Daily volatility: {returns.std()*100:.3f}%")
print(f"  Annualized return: {returns.mean()*252*100:.1f}%")
print(f"  Annualized vol: {returns.std()*np.sqrt(252)*100:.1f}%")

## Part 1: Value at Risk (VaR)

In [ ]:
# ============================================================
# VALUE AT RISK
# ============================================================

print("VALUE AT RISK (VaR)")
print("="*60)

def calculate_var(returns, confidence=0.95, method='historical'):
    """
    Calculate Value at Risk.
    
    Args:
        returns: Array of returns
        confidence: Confidence level (e.g., 0.95 for 95%)
        method: 'historical', 'parametric', or 'cornish_fisher'
    """
    alpha = 1 - confidence
    
    if method == 'historical':
        var = -np.percentile(returns, alpha * 100)
    
    elif method == 'parametric':
        # Assumes normal distribution
        mu = returns.mean()
        sigma = returns.std()
        var = -(mu + sigma * stats.norm.ppf(alpha))
    
    elif method == 'cornish_fisher':
        # Adjusts for skewness and kurtosis
        mu = returns.mean()
        sigma = returns.std()
        skew = stats.skew(returns)
        kurt = stats.kurtosis(returns)
        
        z = stats.norm.ppf(alpha)
        z_cf = (z + (z**2 - 1)*skew/6 + 
                (z**3 - 3*z)*(kurt-3)/24 - 
                (2*z**3 - 5*z)*skew**2/36)
        
        var = -(mu + sigma * z_cf)
    
    return var

# Calculate VaR at different confidence levels
confidence_levels = [0.90, 0.95, 0.99]

print("\nVaR Comparison:")
print(f"{'Confidence':<12} {'Historical':>12} {'Parametric':>12} {'Cornish-Fisher':>14}")
print("-" * 52)

for conf in confidence_levels:
    var_hist = calculate_var(returns, conf, 'historical')
    var_param = calculate_var(returns, conf, 'parametric')
    var_cf = calculate_var(returns, conf, 'cornish_fisher')
    
    print(f"{conf:.0%}          {var_hist*100:>10.3f}%   {var_param*100:>10.3f}%   {var_cf*100:>12.3f}%")

## Part 2: Conditional VaR (CVaR / Expected Shortfall)

In [ ]:
# ============================================================
# CONDITIONAL VAR (EXPECTED SHORTFALL)
# ============================================================

print("CONDITIONAL VAR (CVaR / Expected Shortfall)")
print("="*60)

def calculate_cvar(returns, confidence=0.95):
    """
    Calculate Conditional VaR (Expected Shortfall).
    
    CVaR = E[X | X <= -VaR]
    Average loss given that loss exceeds VaR.
    """
    alpha = 1 - confidence
    var = calculate_var(returns, confidence, 'historical')
    
    # Average of returns worse than VaR
    cvar = -returns[returns <= -var].mean()
    
    return cvar

print("\nVaR vs CVaR:")
print(f"{'Confidence':<12} {'VaR':>12} {'CVaR':>12} {'Ratio':>10}")
print("-" * 48)

for conf in confidence_levels:
    var = calculate_var(returns, conf, 'historical')
    cvar = calculate_cvar(returns, conf)
    
    print(f"{conf:.0%}          {var*100:>10.3f}%   {cvar*100:>10.3f}%   {cvar/var:>8.2f}x")

print("\n💡 CVaR > VaR because it includes the tail beyond VaR")

## Part 3: Maximum Drawdown

In [ ]:
# ============================================================
# DRAWDOWN ANALYSIS
# ============================================================

print("DRAWDOWN ANALYSIS")
print("="*60)

def calculate_drawdowns(returns):
    """
    Calculate drawdown series and statistics.
    """
    cumulative = np.cumprod(1 + returns)
    running_max = np.maximum.accumulate(cumulative)
    drawdowns = (cumulative - running_max) / running_max
    
    max_dd = drawdowns.min()
    max_dd_idx = drawdowns.argmin()
    
    # Find drawdown periods
    underwater = drawdowns < 0
    
    return {
        'drawdowns': drawdowns,
        'max_drawdown': max_dd,
        'max_dd_idx': max_dd_idx,
        'current_dd': drawdowns[-1],
        'pct_underwater': underwater.mean()
    }

dd_stats = calculate_drawdowns(returns)

print(f"Max Drawdown: {dd_stats['max_drawdown']*100:.2f}%")
print(f"Current Drawdown: {dd_stats['current_dd']*100:.2f}%")
print(f"Time Underwater: {dd_stats['pct_underwater']*100:.1f}%")

In [ ]:
# ============================================================
# VISUALIZE RISK METRICS
# ============================================================

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Return distribution with VaR
axes[0, 0].hist(returns * 100, bins=50, density=True, alpha=0.7, color='steelblue')
var_95 = calculate_var(returns, 0.95, 'historical')
cvar_95 = calculate_cvar(returns, 0.95)
axes[0, 0].axvline(-var_95 * 100, color='orange', linestyle='--', linewidth=2, label=f'VaR 95%: {var_95*100:.2f}%')
axes[0, 0].axvline(-cvar_95 * 100, color='red', linestyle='--', linewidth=2, label=f'CVaR 95%: {cvar_95*100:.2f}%')
axes[0, 0].set_xlabel('Daily Return (%)')
axes[0, 0].set_ylabel('Density')
axes[0, 0].set_title('Return Distribution with VaR', fontweight='bold')
axes[0, 0].legend()

# Cumulative returns with drawdown
ax2 = axes[0, 1].twinx()
axes[0, 1].plot(cumulative, color='steelblue', linewidth=1.5, label='Cumulative')
ax2.fill_between(range(len(dd_stats['drawdowns'])), dd_stats['drawdowns'] * 100, 0, alpha=0.3, color='red')
axes[0, 1].set_xlabel('Days')
axes[0, 1].set_ylabel('Cumulative Return', color='steelblue')
ax2.set_ylabel('Drawdown (%)', color='red')
axes[0, 1].set_title('Returns and Drawdowns', fontweight='bold')

# Rolling VaR
window = 60
rolling_var = pd.Series(returns).rolling(window).apply(lambda x: calculate_var(x, 0.95, 'historical'))
axes[1, 0].plot(rolling_var * 100, color='orange', linewidth=1.5)
axes[1, 0].axhline(var_95 * 100, color='gray', linestyle='--', label=f'Full period: {var_95*100:.2f}%')
axes[1, 0].set_xlabel('Days')
axes[1, 0].set_ylabel('Rolling 95% VaR (%)')
axes[1, 0].set_title('Rolling VaR (60-day)', fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Risk metrics table
axes[1, 1].axis('off')
metrics = [
    ['Metric', '95%', '99%'],
    ['VaR (Historical)', f'{calculate_var(returns, 0.95)*100:.2f}%', f'{calculate_var(returns, 0.99)*100:.2f}%'],
    ['VaR (Parametric)', f'{calculate_var(returns, 0.95, "parametric")*100:.2f}%', f'{calculate_var(returns, 0.99, "parametric")*100:.2f}%'],
    ['CVaR', f'{calculate_cvar(returns, 0.95)*100:.2f}%', f'{calculate_cvar(returns, 0.99)*100:.2f}%'],
    ['Max Drawdown', f'{dd_stats["max_drawdown"]*100:.2f}%', ''],
    ['Sharpe Ratio', f'{np.sqrt(252) * returns.mean() / returns.std():.2f}', '']
]

table = axes[1, 1].table(cellText=metrics, loc='center', cellLoc='center')
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1.2, 2)
axes[1, 1].set_title('Risk Summary', fontweight='bold', y=0.85)

plt.tight_layout()
plt.show()

## Part 4: Dynamic Risk Control

In [ ]:
# ============================================================
# DRAWDOWN-BASED RISK CONTROL
# ============================================================

print("DRAWDOWN-BASED RISK CONTROL")
print("="*60)

def apply_drawdown_control(returns, dd_threshold=-0.10, scale_factor=0.5):
    """
    Scale down position when drawdown exceeds threshold.
    
    When DD > threshold: position *= scale_factor
    """
    n = len(returns)
    controlled_returns = np.zeros(n)
    position_scales = np.ones(n)
    
    cumulative = 1.0
    peak = 1.0
    
    for i in range(n):
        # Current drawdown
        current_dd = (cumulative - peak) / peak
        
        # Scale position based on drawdown
        if current_dd < dd_threshold:
            scale = scale_factor
        else:
            scale = 1.0
        
        position_scales[i] = scale
        controlled_returns[i] = returns[i] * scale
        
        # Update cumulative and peak
        cumulative *= (1 + controlled_returns[i])
        peak = max(peak, cumulative)
    
    return controlled_returns, position_scales

# Apply drawdown control
controlled_returns, scales = apply_drawdown_control(returns, dd_threshold=-0.10, scale_factor=0.3)

# Compare
cum_orig = np.cumprod(1 + returns)
cum_controlled = np.cumprod(1 + controlled_returns)

dd_orig = calculate_drawdowns(returns)
dd_controlled = calculate_drawdowns(controlled_returns)

print(f"{'Metric':<20} {'Original':>12} {'Controlled':>12}")
print("-" * 46)
print(f"{'Final Return':<20} {(cum_orig[-1]-1)*100:>10.1f}%   {(cum_controlled[-1]-1)*100:>10.1f}%")
print(f"{'Max Drawdown':<20} {dd_orig['max_drawdown']*100:>10.1f}%   {dd_controlled['max_drawdown']*100:>10.1f}%")
print(f"{'Sharpe Ratio':<20} {np.sqrt(252)*returns.mean()/returns.std():>10.2f}    {np.sqrt(252)*controlled_returns.mean()/controlled_returns.std():>10.2f}")

In [ ]:
# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(cum_orig, label=f'Original ({(cum_orig[-1]-1)*100:.0f}%)', linewidth=1.5)
axes[0].plot(cum_controlled, label=f'Controlled ({(cum_controlled[-1]-1)*100:.0f}%)', linewidth=1.5)
axes[0].set_xlabel('Days')
axes[0].set_ylabel('Cumulative Return')
axes[0].set_title('Drawdown Control Impact', fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].fill_between(range(len(dd_orig['drawdowns'])), dd_orig['drawdowns']*100, 0, alpha=0.5, label='Original')
axes[1].fill_between(range(len(dd_controlled['drawdowns'])), dd_controlled['drawdowns']*100, 0, alpha=0.5, label='Controlled')
axes[1].axhline(-10, color='red', linestyle='--', label='Threshold')
axes[1].set_xlabel('Days')
axes[1].set_ylabel('Drawdown (%)')
axes[1].set_title('Drawdown Comparison', fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════════╗
║              DAY 4 COMPLETE: RISK MANAGEMENT                     ║
╠══════════════════════════════════════════════════════════════════╣
║  ✓ VaR calculation (historical, parametric, Cornish-Fisher)     ║
║  ✓ CVaR / Expected Shortfall                                    ║
║  ✓ Maximum Drawdown analysis                                    ║
║  ✓ Dynamic drawdown control                                     ║
╚══════════════════════════════════════════════════════════════════╝

Tomorrow: Day 5 - Realistic Backtesting
""")